## Analyse Exploratoire

### Distribution
![Distribution](imgs/Distribution.png)

In [ ]:
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

matplotlib.use("Qt5Agg")

# Charger le CSV
df = pd.read_csv("data/fichier-de-donnees-mixtes.csv")

# Encodage du sexe, smoker, sport_licence en entier 0 ou 1
encoder = LabelEncoder()
df["sexe_enc"] = encoder.fit_transform(df["sexe"])
df["smoker_enc"] = encoder.fit_transform(df["smoker"])
df["sport_licence_enc"] = encoder.fit_transform(df["sport_licence"])
df["fr_enc"] = encoder.fit_transform(df["nationalité_francaise"])

# Catégoriel
df["situation_enc"] = encoder.fit_transform(df["situation_familiale"])
df["region_enc"] = encoder.fit_transform(df["region"])

# Ordinal
ordre = [["aucun", "bac", "bac+2", "master", "doctorat"]]
ordinal_enc = OrdinalEncoder(categories=ordre)
df["niv_etude_enc"] = ordinal_enc.fit_transform(df[["niveau_etude"]])

# Extraction de l'année
df["date_creation_compte"] = pd.to_datetime(df["date_creation_compte"], errors="coerce")
df["annee_creation"] = df["date_creation_compte"].dt.year


# Description du fichier
print(f"Description : {df.describe}")

# Affichage de la distribution des données (les données sont toutes numériques)
plt.figure(figsize=(15, 12))

# Sélection des colonnes numériques uniquement
num_cols = df.select_dtypes(include="number").columns

for i, col in enumerate(num_cols, 1):
    plt.subplot(4, 5, i)     # 4x4
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f"Distribution de {col}")
    plt.xlabel(col)
    plt.ylabel("Fréquence")

plt.tight_layout()
plt.show()

## Heatmap

![Correlation](imgs/Correlation.png)

In [37]:
# Heatmap de corrélation
plt.figure(figsize=(10, 8))
corr = df.corr(numeric_only=True)

sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de corrélation")
plt.show()

## Identifier les valeurs manquantes

Utilisation de missingno

![Colonnes manquantes](imgs/Colonnes.png)
![Colonnes manquantes 2](imgs/Colonnes2.png)

In [28]:
# Visualiser la matrice des valeurs manquantes
msno.matrix(df)
plt.show()

# Visualiser le barplot des colonnes manquantes
msno.bar(df)
plt.show()

# Nombre de lignes avec au moins une valeur manquante
missing_line = df.isna().any(axis=1).sum()
total_line = df.shape[0]
print(f"Nb lignes manquantes : {missing_line} / {total_line}")

Nb lignes manquantes : 8802 / 10000


## Nettoyage des Données

### Identifier les colonnes utiles au résultat




### Supprimer les colonnes non pertinentes

3 colonnes sont partielles, les autres sont complètes. Je fais donc le choix d'éliminer des colonnes plutôt que les lignes avec des données manquantes.
Car si on éliminait les lignes incomplètes, on retirerait 8446 / 10000 lignes.

Cependant, pour rester compatible avec de futures données, je supprime sur condition de pourcentage de données manquantes. Car s'il manque seulement 1 données dans une colonnes on ne va pas supprimer la colonne pour autant !


In [29]:
cols_before = df.shape[1]
threshold = 0.20  # 20%
df_clean = df.drop(columns=df.columns[df.isna().mean() > threshold])
cols_after = df_clean.shape[1]
print(f"Nb colonnes avant nettoyage : {cols_before}, nb colonnes après : {cols_after}")


Nb colonnes avant nettoyage : 24, nb colonnes après : 20


Au cas où il resterait des lignes avec des champs manquants, je supprime les lignes concernée. Il n'y en a pas actuellement dans le jeu de données, mais cela pourrait être le cas dans le futur.

In [30]:
lines_before = df_clean.shape[0]
# Supprimer les lignes contenant des valeurs manquantes
df_clean = df_clean.dropna()
lines_after = df_clean.shape[0]
print(f"Nb lignes avant nettoyage : {lines_before}, nb lignes après : {lines_after}")

Nb lignes avant nettoyage : 10000, nb lignes après : 10000


### Ecarter les outliers
Filtrer les outliers avec la méthode IQR (sur toutes les colonnes)

In [31]:
# Sélection des colonnes numériques uniquement
df_num = df_clean.select_dtypes(include="number")

lines_before = df_num.shape[0]
Q1 = df_num.quantile(0.25)
Q3 = df_num.quantile(0.75)
IQR = Q3 - Q1

df_filtered = df_num[~((df_num < (Q1 - 1.5 * IQR)) | (df_num > (Q3 + 1.5 * IQR))).any(axis=1)]

lines_after = df_filtered.shape[0]
print(f"Nb lignes avant filtrage des outliers : {lines_before}, nb lignes après : {lines_after}")


Nb lignes avant filtrage des outliers : 10000, nb lignes après : 9751


### Imputation
Pas d'imputation dans ce cas, car une fois les colonnes quasi-vides enlevées, il n'y a pas de ligne avec des valeurs nulles. 

### Catégorisation / Normalisation

Toutes les colonnes sont de types numériques, donc pas besoin des catégoriser. Il faudra par contre les normaliser.

#### Standardisation des données
La standardisation consiste à transformer chaque variable pour qu’elle ait une moyenne de 0 et un écart-type de 1.  
Cela permet de mettre toutes les colonnes numériques sur la même échelle, ce qui est essentiel pour de nombreux algorithmes de Machine Learning.




In [32]:
from sklearn.preprocessing import StandardScaler

# Initialiser le scaler
scaler = StandardScaler()

# Colonnes numériques (toutes celles de df_filtered)
num_cols = df_filtered.select_dtypes(include="number").columns

# Copie du dataset
df_standardized = df_filtered.copy()

# Application de la standardisation
df_standardized[num_cols] = scaler.fit_transform(df_filtered[num_cols])

print("Standardisation effectuée (aperçu) :")
display(df_standardized.head())

# Sauvegarde du dataset standardisé
output_path = "data/dataset_standardized.csv"
df_standardized.to_csv(output_path, index=False)

print(f"Dataset sauvegardé sous : {output_path}")



Standardisation effectuée (aperçu) :


,age,taille,poids,revenu_estime_mois,risque_personnel,montant_pret,sexe_enc,smoker_enc,sport_licence_enc,fr_enc,situation_enc
0,1.571914,-0.910330,-0.192590,-1.447571,-1.338992,0.429579,1.001129,-1.008342,-0.969195,0.815904,-1.450626
1,-0.151545,-0.184470,0.327662,2.425334,1.002711,2.312510,1.001129,-1.008342,-0.969195,0.815904,-1.450626
2,1.453054,-0.992117,-1.684891,1.142898,-1.270118,0.900855,1.001129,0.991727,-0.969195,0.815904,-1.450626
3,0.918188,-0.828543,1.203875,0.700709,-0.615819,0.708102,-0.998873,0.991727,1.031784,0.815904,-0.757003
4,-1.696715,0.817420,0.512488,1.232041,0.555033,0.163597,-0.998873,-1.008342,1.031784,0.815904,-1.450626


Dataset sauvegardé sous : data/dataset_standardized.csv


In [33]:
# Heatmap de corrélation sur les données nettoyées et standardisées
plt.figure(figsize=(10, 8))
corr = df_standardized.corr(numeric_only=True)

sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de corrélation")
plt.show()